#### 로지스틱 분류 LogisticRegression

일반 선형 회귀 모형 ax + b => 0과 1로 구분되는 데이터셋은 회귀분석으로는 힘듬 

=> 오즈비를 활용한 ln(p/1-p) = ax+b  => p = 시그모이드 함수 

=> p 임계점에 따라 0과 1의 값으로 할당 가능 

=> 즉 로지스틱 모델은 다양한 변수들을 종합하여 1,0 두가지로 확률을 예측하여 분류 하는 방법

#### 오즈(Odds)란?

성공 확률이 실패 확률에 비해 몇 배 더 높은가를 나타냄

<img src='./images/odds.png' align='left'>
<br/><br/><br/><br/>

#### 로짓(logit)이란?

오즈 비에 로그를 씌운값

<img src='./images/logits.png' align='left'>
<br/><br/><br/><br/>

#### 로지스틱 유도 

로짓과 회귀식의 연관성

<img src='./images/logistic.png' align='left'>
<br/><br/><br/><br/>

이런뒤 p를 기준으로 우변 정렬하면 로지스틱 펑션이 나옴

<img src='./images/logistic-function.png' align='left'>
<br/><br/><br/><br/>

In [16]:
# sklearn 이용
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=7777)
train_y

# C : Inverse of regularization, 절편없으면: fit_intercept=False
lor = LogisticRegression(penalty='l2', C=100000, random_state=98709, solver='newton-cg')
lor.fit(train_x, train_y)
# 예측
preds = lor.predict(test_x)

# 예측 확률
preds = lor.predict_proba(test_x)

# logit 로짓 값들의 평균
# observation별 logit 값의 평균
# odds 는 실패에 비해 성공할 확률의 비 : p/(1-p)
# odds 에 Log를 취한값이 바로 로짓 : Log(p)
# 회귀분석의 y =ax +b 가 ln(p/1-p) = ax+b 가 된것 => 즉, 로지스틱 회귀분석은 일반회귀 함수를 로짓으로 변형한 분석법
import math
import numpy as np
arr = []
for pred in preds:
  # 만약에 아주 확률이 1에 가까워 1 이 나온다면 최대한 1보다 작은 가장 큰값 입력
  arr.append(math.log(float(pred[1])/float(1-pred[1])))
print(np.mean(arr))
# 또는
print(np.mean(np.log(preds[:,1]/preds[:,0])))

# 가장 영향이 큰(기준: Odds Ratio) 독립변수 => e 의 지수에 해당 변수가 위치하므로 np.exp 를 사용한다.
cols = breast_cancer.feature_names
print(cols[np.exp(lor.coef_).argmax()])

# 변수에 대한 계수
lor.coef_

# 특정 변수에 대한 오즈비 => 로지스틱 회귀분석에서 오즈비는 np.exp(특정변수의 계수)
np.exp(lor.coef_[0])

# 평균면적이 1 증가할때 마다 유방암에 걸릴 확률이 몇배나 증가한다고 할 수 있는지를 구하라 => mean area 에 대한 오즈비를 구하라(Odds Ratio)
# 오즈비는 1증가할떄마다 얼마나 증가하는 지를 보인다. 만약 0,2가 값으로 있다면 minmax를 하는것이 좋고, 3가지 값이 있다면 one-hot encoder 후 학습시킨뒤 비를 구하면 더 정확함
i = cols.tolist().index('mean area')
np.exp(lor.coef_[0][i])

# 예측확률에 따른 예측치 결정(threshold or cut off value)
pred_pro = np.where(preds[:,1]>0.5, 1, 0)
pred_pro

-3.129580575362974
-3.129580575362974
concavity error


C:\Users\carl\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [17]:
# 통계적으로 유의한 독립변수 찾는법 => statsmodels 사용
from sklearn import datasets
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target
cols = [i.replace(' ' , '_') for i in breast_cancer.feature_names] + ['y']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.4, random_state=7777)
train = np.hstack((train_x, train_y.reshape(-1,1)))
test = np.hstack((test_x, test_y.reshape(-1,1)))
train = pd.DataFrame(train, columns=cols)
test = pd.DataFrame(test, columns=cols)

# 20 개 넘어가면 잘안됨
formula = 'y ~ mean_radius + mean_texture + mean_perimeter + mean_area + mean_smoothness + mean_compactness + mean_concavity'
# train
lor = sm.Logit.from_formula(formula, train).fit()
# 요약
print(lor.summary())
# p_value 확인 => 0.05 이하이면 의미있는 변수
lor.pvalues
# 계수확인
lor.params

# 또 다른 방법. 일반선형 모델인 GLM(Generalize linear Model) 에서 이진 분류 하면 link function이 Logit 되어 Logistic regression 됨
import statsmodels.formula.api as smf
smf.glm(formula, train, family=sm.families.Binomial()).fit().summary()


Optimization terminated successfully.
         Current function value: 0.109327
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  341
Model:                          Logit   Df Residuals:                      333
Method:                           MLE   Df Model:                            7
Date:                Sat, 19 Sep 2020   Pseudo R-squ.:                  0.8328
Time:                        15:56:50   Log-Likelihood:                -37.281
converged:                       True   LL-Null:                       -222.95
Covariance Type:            nonrobust   LLR p-value:                 3.307e-76
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           45.0582     17.537      2.569      0.010      10.686      79.430
mean_radiu

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  341
Model:                            GLM   Df Residuals:                      333
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -37.281
Date:                Sat, 19 Sep 2020   Deviance:                       74.561
Time:                        15:56:50   Pearson chi2:                     450.
No. Iterations:                     9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           45.0582     17.537      2.569      0.010      10.686      79.430
mean_radius         -5.8674      6.405     -0.916      0.360     -18.422       6.687
mean_texture        -0.4743      0.101     -4.707      0.000      -0.672      -0.277
mean_perimeter       0.7831      0.808      0.970      0.332      -0.800       2.366
mean_area           -0.0112      0.023     -0.494      0.621      -0.055       0.033
mean_smoothness   -139.7322     37.109     -3.765      0.000    -212.465     -67.000
mean_compactness   -15.3550     27.752     -0.553      0.580     -69.748      39.038
mean_concavity     -22.8645      8.099     -2.823      0.005     -38.739      -6.990
====================================================================================
"""